# テストデータによる評価

## 評価(test)<a name="test"></a>

- 過学習した状態で機械学習プロダクトを本番環境にデプロイすると痛い目を見るので、モデルが過学習していないかを評価するのは重要
- 評価には通常、訓練データとは別のテストデータを用意して、訓練済みモデルを適用する
- 一貫した評価のため、テストデータには評価する対象のモデル間で毎回同じデータセットを用いる
- テストデータに偏りがあると正しい評価ができないため、慎重に設計する必要がある
- データセットを訓練データとテストデータで6:4、7:3、8:2のいずれかで分割して使用するのが一般的
- データセットが十分に大きい場合は、9:1や99:1も用いられる

## 精度による評価<a name="accuracy"></a>

分類問題の場合、精度(正解率)が最も単純な評価指標。

### テストデータの作成<a name="accuracy_data"></a>

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

loader = load_breast_cancer()
X, y = loader.data, loader.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

### 評価実行<a name="accuracy_test"></a>

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='rbf', C=1).fit(X_train, y_train)
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)
print('Train accuracy: {train:.3f}, Test accuracy: {test:.3f}'.format(train=train_score, test=test_score))

## 誤差による評価<a name="error"></a>

回帰問題の場合、精度では評価できない。予測値と正解の誤差が最も単純な評価指標だが、外れ値の影響を受けやすい。

### テストデータの作成<a name="error_test"></a>

In [ ]:
from sklearn.datasets import load_diabetes

loader = load_diabetes()
X, y = loader.data, loader.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

### 学習<a name="error_training"></a>

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(X_train, y_train)
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

### 二乗平均平方根(Root Mean Square, RMS)<a name="root_mean_square"></a>

回帰に通常使われる平均二乗誤差の場合、その平方根が目安になる。平均二乗誤差については[別の項](../optimization/optimization.ipynb#mean_squared_error)で説明する。

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

train_score = np.sqrt(mean_squared_error(y_train, train_pred))
test_score = np.sqrt(mean_squared_error(y_test, test_pred))
print('Train RMS: {train:.3f}, Test RMS: {test:.3f}'.format(train=train_score, test=test_score))

### 誤差のプロット<a name="error_plot"></a>

誤差(残差)をプロットしてみて、正解の周辺にランダムに予測値が分布しているのが理想的。誤差に規則性がある場合は、何らかの特徴を補足しきれていない可能性がある。

In [ ]:
import matplotlib.pyplot as plt

train_residual = train_pred - y_train
test_residual = test_pred - y_test
all_pred = np.concatenate((train_pred, test_pred), axis=0)
all_residual = np.concatenate((train_residual, test_residual), axis=0)
margin = 10
x_min, x_max = all_pred.min() - margin, all_pred.max() + margin
y_min, y_max = all_residual.min() - margin, all_residual.max() + margin

plt.scatter(train_pred, train_pred - y_train, c='blue', label='Train')
plt.scatter(test_pred, test_pred - y_test, c='red', label='Test')
plt.hlines(y=0, xmin=x_min, xmax=x_max, color='green')

plt.legend(loc='upper left')
plt.xlabel('Prediction')
plt.ylabel('Residual')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

plt.show()

## 決定係数による評価<a name="r2"></a>

回帰問題へのモデルの適合度を表す指標として、決定係数$R^2$がある。$R^2$は、正解データの分散をどれくらい補足できているかの割合を示す。モデルの性能があまりに悪いと、負の値も取りうる。

In [ ]:
from sklearn.metrics import r2_score

train_r2 = r2_score(y_train, train_pred)
test_r2 = r2_score(y_test, test_pred)
print('Train R^2: {train:.2f}, Test R^2: {test:.2f}'.format(train=train_r2, test=test_r2))